# Topological Analysis of Circuit Subvolumes

Here we discuss the pipeline that we use to run a topological analysis.

To interact with the application all you need to do is provide a configuration,
that we discuss in a separate notebook.
Here we discuss how this configuration can be used to launch a pipeline.

In [4]:
from importlib import reload
from pathlib import Path
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sbn
from connsense import pipeline
from connsense.io import logging

LOG = logging.get_logger("Toological analysis tutorial")
proj83 = Path("/gpfs/bbp.cscs.ch/project/proj83")
path_results = (proj83 
                /"home/sood/analyses/manuscript/topological-analysis-subvolumes"
                /"notebooks" / "results")

We have implemented a *class* to handle the running of the pipeline that
can be defined using path to a configuration.

In [6]:
from connsense import pipeline
from connsense.io import read_config
path_config = path_results/"develop-config.json"
config = read_config.read(path_config)
topaz = pipeline.TopologicalAnalysis(path_config)

The pipeline has a state that is used to run the steps.

In [4]:
LOG.info("Initialized a topological analysis pipeline to run:")
for i, step in enumerate(topaz.state.queue):
    LOG.info("\t%s. %s", i, step)

 2021-11-10 20:52:34,753: Initialized a topological analysis pipeline to run:
 2021-11-10 20:52:34,756: 	0. define-subtargets
 2021-11-10 20:52:34,757: 	1. extract-neurons
 2021-11-10 20:52:34,758: 	2. extract-connectivity
 2021-11-10 20:52:34,759: 	3. randomize-connectivity
 2021-11-10 20:52:34,760: 	4. analyze-connectivity


In [5]:
working_dir = proj83 / "scratch" / "manuscript" / "topological-analysis"
output = working_dir / "results" / "topological-sampling.h5"

q = topaz.run(steps=["define-subtargets", "extract-neurons", 
                     "extract-connectivity", "randomize-connectivity"],
              output=output, dry_run=True)

 2021-11-10 20:52:35,702: Dispatch from 5 queue: ['define-subtargets', 'extract-neurons', 'extract-connectivity', 'randomize-connectivity', 'analyze-connectivity']
 2021-11-10 20:52:35,702: Dispatch pipeline step define-subtargets
 2021-11-10 20:52:35,705: Get subtargets for config {'paths': {'circuit': {'Bio_M': '/gpfs/bbp.cscs.ch/project/proj83/circuits/Bio_M/20200805/CircuitConfig_TC_WM'}, 'flatmap': {}, 'define-subtargets': ('/gpfs/bbp.cscs.ch/project/proj83/home/sood/analyses/manuscript/define_subtargets/notebooks/results/topological_sampling.h5', 'subtargets'), 'extract-neurons': ('/gpfs/bbp.cscs.ch/project/proj83/home/sood/analyses/manuscript/define_subtargets/notebooks/results/topological_sampling.h5', 'neurons'), 'evaulate-subtargets': ('/gpfs/bbp.cscs.ch/project/proj83/home/sood/analyses/manuscript/define_subtargets/notebooks/results/topological_sampling.h5', 'subtarget_quality'), 'extract-connectivity': ('/gpfs/bbp.cscs.ch/project/proj83/home/sood/analyses/manuscript/define_

 2021-11-10 20:52:35,738: 	(3). randomize-connectivity: Result saved (PosixPath('/gpfs/bbp.cscs.ch/project/proj83/scratch/manuscript/topological-analysis/results/topological-sampling.h5'), 'con_mats/randomized')


## Analysis layer.

In [8]:
config["paths"]["analyze-connectivity"]

('/gpfs/bbp.cscs.ch/project/proj83/home/sood/analyses/manuscript/topological-analysis-subvolumes/notebooks/results/topological_sampling.h5',
 'analysis')

In [17]:
from connsense.io.write_results import read_toc_plus_payload
path_extract_connectivity = config["paths"]["extract-connectivity"]
LOG.info("load connectivity from %s", path_extract_connectivity)
original = read_toc_plus_payload(path_extract_connectivity, "extract-connectivity")
toc_orig = pd.concat([original], keys=["original"], names=["algorithm"])

LOG.info("Total subtargets: %s", toc_original.shape[0])

toc_orig.head()

 2021-11-11 13:42:26,993: load connectivity from ('/gpfs/bbp.cscs.ch/project/proj83/home/sood/analyses/manuscript/topological-analysis-subvolumes/notebooks/results/topological_sampling.h5', 'con_mats/original')
 2021-11-11 13:42:27,028: Total subtargets: 247


algorithm  circuit  connectome  subtarget  flat_x       flat_y
original   Bio_M    local       R0;C10     3983.716857  0.0       <connsense.io.write_results.LazyMatrix object ...
                                R0;C11     4382.088543  0.0       <connsense.io.write_results.LazyMatrix object ...
                                R0;C12     4780.460229  0.0       <connsense.io.write_results.LazyMatrix object ...
                                R0;C2      796.743371   0.0       <connsense.io.write_results.LazyMatrix object ...
                                R0;C3      1195.115057  0.0       <connsense.io.write_results.LazyMatrix object ...
Name: gids, dtype: object

In [13]:
path_randomized_connectivity = config["paths"]["randomize-connectivity"]
LOG.info("load connectivity from %s", path_randomized_connectivity)
toc_randomized = read_toc_plus_payload(path_randomized_connectivity,
                                       "randomize-0jconnectivity")
LOG.info("Total subtargets: %s", toc_randomized.shape[0])


 2021-11-11 13:39:24,367: load connectivity from ('/gpfs/bbp.cscs.ch/project/proj83/home/sood/analyses/manuscript/topological-analysis-subvolumes/notebooks/results/topological_sampling.h5', 'con_mats/randomized')
 2021-11-11 13:39:24,490: Total subtargets: 494


In [19]:
sample = 0.01
S = np.float(sample)
sample = toc_original.sample(frac=S).index
sample.to_frame()

/gpfs/bbp.cscs.ch/ssd/slurmTmpFS/sood/4729/ipykernel_240966/3876114226.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  S = np.float(sample)


circuit connectome subtarget  \
circuit connectome subtarget flat_x      flat_y                                
Bio_M   local      R5;C11    4581.274386 1725.0   Bio_M      local    R5;C11   
                   R11;C5    2191.044272 3795.0   Bio_M      local    R11;C5   

                                                      flat_x  flat_y  
circuit connectome subtarget flat_x      flat_y                       
Bio_M   local      R5;C11    4581.274386 1725.0  4581.274386  1725.0  
                   R11;C5    2191.044272 3795.0  2191.044272  3795.0

In [34]:
def get(i, g):
    print(i)
    g = g.droplevel("algorithm")
    return pd.concat([g.loc[sample]], keys=[i], names=["algorithm"])
    
q = pd.concat([get(i, g) for i, g in toc_randomized.groupby("algorithm")])

Erodos-Renyi-controlling-in-degree
Erodos-Renyi-controlling-out-degree


In [35]:
q

algorithm                            circuit  connectome  subtarget  flat_x       flat_y
Erodos-Renyi-controlling-in-degree   Bio_M    local       R5;C11     4581.274386  1725.0    <connsense.io.write_results.LazyMatrix object ...
                                                          R11;C5     2191.044272  3795.0    <connsense.io.write_results.LazyMatrix object ...
Erodos-Renyi-controlling-out-degree  Bio_M    local       R5;C11     4581.274386  1725.0    <connsense.io.write_results.LazyMatrix object ...
                                                          R11;C5     2191.044272  3795.0    <connsense.io.write_results.LazyMatrix object ...
dtype: object

In [ ]:
ce